In [6]:
from __future__ import print_function, absolute_import

import os 
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torchvision
import torchvision.datasets as datasets
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageChops
import scipy.misc
import sys, math
sys.path.append('..')

import scripts.utils
from scripts.utils.logger import Logger, savefig
from scripts.utils.evaluation import accuracy, AverageMeter, final_preds
from scripts.utils.misc import save_checkpoint, save_pred, adjust_learning_rate
from scripts.utils.osutils import mkdir_p, isfile, isdir, join
from scripts.utils.imutils import batch_with_heatmap,normalize_batch,im_to_numpy
from scripts.utils.transforms import fliplr, flip_back
import scripts.models as models
import scripts.datasets as datasets
import random
from matplotlib import cm

class objectview(object):
    def __init__(self, *args, **kwargs):
        d = dict(*args, **kwargs)
        self.__dict__ = d


dataset_name= 'data'
dataroot = '/home/oishii/Documents/DeepHarmonization/'

data_config  = objectview({'input_size':256,'normalized_input':False,'data_augumentation':False,'withseg':False})
sample = []

# run with gan-based method            
with torch.no_grad():
        val_loader = torch.utils.data.DataLoader(datasets.COCO(dataroot,dataset_name,config=data_config,sample=sample,gan_norm=False),
                                                 batch_size=1, shuffle=False,
                                                 num_workers=2, pin_memory=False)
        checkpoint_dict = torch.load('/home/oishii/Documents/deep-harimonization-improved/psnr/cocov4/1e3_bs8_256_2017_radhnv3xgaussian/model_best.pth.tar')
            
        for i, (inputs, target) in enumerate(val_loader):            
            
            model = models.__dict__['radhnv3xgaussian']()
            model.load_state_dict(checkpoint_dict['state_dict'])
            model.eval()

            inputs = inputs
            target = target[0]
            
            output = model(inputs)
            
            inputsnp = (im_to_numpy(inputs[0,0:3])*255).clip(0,255).astype(np.uint8)
            outputnp = (im_to_numpy(output[0])*255).clip(0,255).astype(np.uint8)
            targetnp = (im_to_numpy(target[0])*255).clip(0,255).astype(np.uint8)
            masknp = (im_to_numpy(inputs[0,3:4].repeat(3,1,1))*255).clip(0,255).astype(np.uint8)
            
            im_output = Image.fromarray(outputnp)
            
            im_input = Image.fromarray(inputsnp)
            
            im_target = Image.fromarray(targetnp)
            
            im_mask = Image.fromarray(masknp)
            
            im_output.save(str(i)+'.png')
#             im_input.save('input_'+str(i)+'.png')
#             im_target.save('target_'+str(i)+'.png')
#             im_mask.save('mask_'+str(i)+'.png')
            
#             plt.figure(figsize=(20,5))
#             plt.xlabel(resume)
#             plt.imshow(np.concatenate((inputsnp,outputnp,targetnp,masknp,jetnp),axis=1))
#             plt.draw()
            
        

total Dataset of data is :  31
